In [1]:
import pandas as pd
import numpy as np

BASE = "../data/raw/tabular/nces"

/tmp/ipykernel_16150/2973593007.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
directory = pd.read_csv(f"{BASE}/directory.csv")
staff = pd.read_csv(f"{BASE}/staff.csv")
lunch = pd.read_csv(f"{BASE}/lunch.csv")

# IMPORTANT: characteristics needs header=0, NOT header=1
characteristics = pd.read_csv(f"{BASE}/characteristics.csv", header=0)

scores = pd.read_csv("../data/processed/esa_fca_scores_cleaned.csv")

In [3]:
print("DIRECTORY:", "NCESSCH" in directory.columns)
print("STAFF:", "NCESSCH" in staff.columns)
print("LUNCH:", "NCESSCH" in lunch.columns)
print("CHARACTERISTICS:", "NCESSCH" in characteristics.columns)

DIRECTORY: True
STAFF: True
LUNCH: True
CHARACTERISTICS: True


In [4]:
directory[["SCH_NAME", "LEA_NAME"]].head()

,SCH_NAME,LEA_NAME
0,ANDERSON H S,AUSTIN ISD
1,AUSTIN H S,AUSTIN ISD
2,CROCKETT ECHS,AUSTIN ISD
3,NAVARRO EARLY COLLEGE H S,AUSTIN ISD
4,MCCALLUM H S,AUSTIN ISD


In [5]:
# Force NCESSCH to be string everywhere (CRITICAL FIX)
for df_ in [directory, staff, lunch, characteristics]:
    if "NCESSCH" in df_.columns:
        df_["NCESSCH"] = df_["NCESSCH"].astype(str)

print("NCESSCH types fixed")

NCESSCH types fixed


In [6]:
# --- DIRECTORY (base table) ---
directory_base = directory[
    [
        "NCESSCH",
        "SCH_NAME",
        "LEA_NAME",
        "MCITY",
        "MZIP"
    ]
].copy()

# --- STAFF ---
staff_base = staff[
    [
        "NCESSCH",
        "TEACHERS"
    ]
].copy()

# --- LUNCH ---
lunch_base = lunch[
    [
        "NCESSCH",
        "LUNCH_PROGRAM",
        "STUDENT_COUNT"
    ]
].copy()

print("Column trimming complete")


Column trimming complete


In [16]:
lunch_base["STUDENT_COUNT"] = pd.to_numeric(
    lunch_base["STUDENT_COUNT"], errors="coerce"
)

lunch_agg = (
    lunch_base
    .groupby("NCESSCH", as_index=False)["STUDENT_COUNT"]
    .sum()
    .rename(columns={"STUDENT_COUNT": "LUNCH_ELIGIBLE"})
)

In [18]:
scores.columns
scores.head()

,School,ESA_Score,FCA_Score
0,Akins ECHS,59,80
1,Anderson High School,61,76
2,Austin High School,59,70
3,Bowie High School,62,65
4,Crockett ECHS,55,52


In [10]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [17]:
target = "ESA_Score"

feature_cols = [
    "TEACHERS",
    "MZIP",
]

df_model = df[feature_cols + [target]].copy()
df_model

,TEACHERS,MZIP,ESA_Score
0,123.33,78759,61.0
1,126.63,78703,59.0
2,98.29,78745,55.0
3,109.13,78758,NaN
4,102.70,78756,50.0
5,66.53,78752,NaN
6,76.76,78704,NaN
7,154.93,78749,62.0
8,25.47,78702,NaN
9,159.71,78748,NaN


In [18]:
df_train = df_model[df_model[target].notna()]
df_missing = df_model[df_model[target].isna()]

print("Train rows:", len(df_train))
print("Missing rows:", len(df_missing))

Train rows: 6
Missing rows: 6


In [25]:
rf = RandomForestRegressor(
    n_estimators=200,
    random_state=42
)

rf.fit(X_train, y_train)

RandomForestRegressor(n_estimators=200, random_state=42)

In [26]:
rf.fit(X, y)
df.loc[df[target].isna(), "ESA_Predicted"] = rf.predict(X_missing)

In [30]:
X_missing = df_missing[feature_cols]
df["ESA_Predicted"] = rf.predict(df[feature_cols])

In [31]:
df[[
    "SCH_NAME",
    "TEACHERS",
    "ESA_Score",
    "ESA_Predicted"
]]

,SCH_NAME,TEACHERS,ESA_Score,ESA_Predicted
0,ANDERSON H S,123.33,61.0,58.565
1,AUSTIN H S,126.63,59.0,59.045
2,CROCKETT ECHS,98.29,55.0,55.060
3,NAVARRO EARLY COLLEGE H S,109.13,NaN,53.570
4,MCCALLUM H S,102.70,50.0,53.295
5,NORTHEAST EARLY COLLEGE H S,66.53,NaN,55.550
6,TRAVIS EARLY COLLEGE H S,76.76,NaN,57.175
7,BOWIE H S,154.93,62.0,60.655
8,GARZA INDEPENDENCE H S,25.47,NaN,57.695
9,AKINS H S,159.71,NaN,60.655


In [48]:
print(df.columns.tolist())

['NCESSCH', 'SCH_NAME', 'LEA_NAME', 'MCITY', 'MZIP', 'TEACHERS', 'LUNCH_ELIGIBLE', 'school_key', 'ESA_Score', 'FCA_Score', 'ESA_Predicted', 'FCA_Predicted']


In [54]:
# --- FCA setup ---
target = "FCA_Score"

feature_cols = [
    "TEACHERS",
]

df_known = df[df[target].notna()].copy()
df_missing = df[df[target].isna()].copy()

X_known = df_known[feature_cols]
y_known = df_known[target]

print("Known FCA samples:", len(df_known))
print("Missing FCA samples:", len(df_missing))
print(df_known[["SCH_NAME", target] + feature_cols].head())

Known FCA samples: 6
Missing FCA samples: 6
        SCH_NAME  FCA_Score  TEACHERS
0   ANDERSON H S       76.0    123.33
1     AUSTIN H S       70.0    126.63
2  CROCKETT ECHS       52.0     98.29
4   MCCALLUM H S       19.0    102.70
7      BOWIE H S       65.0    154.93


In [55]:
from sklearn.ensemble import RandomForestRegressor

rf_fca = RandomForestRegressor(
    n_estimators=200,
    random_state=42
)

rf_fca.fit(X_known, y_known)

RandomForestRegressor(n_estimators=200, random_state=42)

In [114]:
df["FCA_Predicted"] = rf_fca.predict(df[feature_cols])

In [115]:
df[["SCH_NAME", "TEACHERS", "FCA_Score", "FCA_Predicted"]] \
  .sort_values("FCA_Predicted", ascending=False)

,SCH_NAME,TEACHERS,FCA_Score,FCA_Predicted
0,ANDERSON H S,123.33,76.0,69.985
1,AUSTIN H S,126.63,70.0,67.420
7,BOWIE H S,154.93,65.0,67.030
9,AKINS H S,159.71,NaN,67.030
5,NORTHEAST EARLY COLLEGE H S,66.53,NaN,64.365
8,GARZA INDEPENDENCE H S,25.47,NaN,64.365
10,LBJ ECHS,51.39,71.0,64.365
6,TRAVIS EARLY COLLEGE H S,76.76,NaN,56.290
2,CROCKETT ECHS,98.29,52.0,47.560
11,LASA H S,82.70,NaN,47.190


In [116]:
def normalize_name(x):
    return (
        x.upper()
        .replace("HIGH SCHOOL", "H S")
        .replace("SCHOOL", "")
        .replace(".", "")
        .strip()
    )

df["SCH_NAME_NORM"] = df["SCH_NAME"].apply(normalize_name)


In [117]:
df = df.drop(columns=["ESA_text", "FCA_text"], errors="ignore")

df = df.merge(
    text_preds,
    on="SCH_NAME_NORM",
    how="left"
)

In [118]:
df[["SCH_NAME", "ESA_text", "FCA_text"]]

,SCH_NAME,ESA_text,FCA_text
0,ANDERSON H S,1.0,2.0
1,AUSTIN H S,1.0,2.0
2,CROCKETT ECHS,1.0,1.0
3,NAVARRO EARLY COLLEGE H S,NaN,NaN
4,MCCALLUM H S,1.0,0.0
5,NORTHEAST EARLY COLLEGE H S,NaN,NaN
6,TRAVIS EARLY COLLEGE H S,NaN,NaN
7,BOWIE H S,1.0,1.0
8,GARZA INDEPENDENCE H S,NaN,NaN
9,AKINS H S,NaN,NaN
